#### Imports

In [ ]:
import pandas as pd
import numpy as np
import os
import typing
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta

from tensorflow.keras.regularizers import l2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import timeseries_dataset_from_array

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanAbsoluteError, MeanSquaredError, MeanSquaredLogarithmicError

from extraction.extract import *
from extraction.extractionvalues import *
from extraction.extractadjacency import getAdjacencyMatrix, distance_weight_adjacency
# from extraction.adj_data import *

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler
%load_ext tensorboard
# %tensorboard --logdir logs/fit

##### Settings

These settings control the jupyter notebook. We have generated and stored 2 models, one for the top10 airports and one the top50. The default settings loads the model for the top 50 and displays the results. To try the top10 model, change airportsFull to ICAOTOP10 and runName to "top10MSE". To try a custom run, choose an airport list of your liking, change runName to something else, change saveModel to True, and loadModel to False. For reference, top10 took about 15 minutes to train and top50 about 45 minutes. 

Once your settings are set, you can just click run all in the jupyter interface. In the final plotting cell more settings can be changed.

In [ ]:
airportsFull = ICAOTOP50
airports = airportsFull[::]
airports.remove("LTFM")  # removing the new istanbul airport as it opened mid-2019

n_nodes = n_airports = len(airports)
start = datetime(2018, 3, 1)
end = datetime(2019, 1, 1)
timeslotLength = 30

# Run settings
batch_size = 64
epochs = 1500
patience = 100
input_sequence_length = 4 * int(60/timeslotLength)
forecast_horizon = 10
multi_horizon = True

learning_rate = 0.0001 
# learning_rate = 0.0005


runName = "top50MSE30minsV6"
saveModel = False
loadModel = True
modelSaveFolder = "kerasModels/"
modelSaveLocation = modelSaveFolder + runName


##### Load Data

In [ ]:
dataDict = generateNNdataMultiple(
    airports, timeslotLength, GNNFormat=True, start=start, end=end, disableWeather=True
)
times = list(dataDict.values())[0]["T"]


##### Process Data

We reduce the amount of features and normalise the data

In [ ]:
# we remove some of the most uncorrelated features
columnsToDrop = [
    "weekend",
    "winter",
    "spring",
    "summer",
    "autumn",
    "night",
    "morning",
    "afternoon",
    "planes",
]

Xlist = []
Ylist = []

# scaler = MinMaxScaler(feature_range=(-1, 1))
# scaler = RobustScaler()
for airport in airports:
    # T x F
    X = dataDict[airport]["X"].drop(columnsToDrop, axis=1).to_numpy()
    # X = scaler.fit_transform(X)
    Xlist.append(X)

    Y = dataDict[airport]["Y"].iloc[:, :].to_numpy()
    Ylist.append(Y)


Xlist = np.stack(Xlist)
Ylist = np.stack(Ylist)
# N x T x F
Xarray = np.swapaxes(Xlist, 0, 1)
Yarray = np.swapaxes(Ylist, 0, 1)

# Reshape to a flat array that goes arrival then departure delay
# Yarray = np.reshape(Yarray, newshape=[len(times), len(airports)*2], order="F")
# # T x N x F

# # Normalise over the features
Xmean, Xstd = Xarray.mean(axis=0), Xarray.std(axis=0)
Xarray = (Xarray - Xmean) / Xstd

print("T x N x F: ", "Xarray =", Xarray.shape, "|", "Yarray =", Yarray.shape)


Here we make the raw data splits

In [ ]:
train_split, val_split = 0.6, 0.25

fullLength = len(times)
train_idx = int(train_split * fullLength)
val_idx = int((val_split + train_split) * fullLength)
print(
    f"Train split: {0}:{train_idx} | Validation split: {train_idx}:{val_idx} | Test split: {val_idx}:{fullLength}"
)

# generate raw splits
Xtrain, Xval, Xtest = Xarray[0:train_idx], Xarray[train_idx:val_idx], Xarray[val_idx::]
Ytrain, Yval, Ytest = Yarray[0:train_idx], Yarray[train_idx:val_idx], Yarray[val_idx::]

# Save test timeslots for plotting purposes
testTimes = times.iloc[val_idx::, 0].tolist()


##### Create tensorflow dataset

Create a tensorflow dataset that handles lookback, lookforward and multi-horizon properties. Retrieved from a keras example and modified: https://keras.io/examples/timeseries/timeseries_traffic_forecasting/

In [ ]:
def create_tf_dataset(
    data_array: np.ndarray,
    target_array,
    input_sequence_length: int,
    forecast_horizon: int,
    batch_size: int = 128,
    shuffle=True,
    multi_horizon=False,
):
    """Creates tensorflow dataset from numpy array.

    This function creates a dataset where each element is a tuple `(inputs, targets)`.
    `inputs` is a Tensor
    of shape `(batch_size, input_sequence_length, num_routes, 1)` containing
    the `input_sequence_length` past values of the timeseries for each node.
    `targets` is a Tensor of shape `(batch_size, forecast_horizon, num_routes)`
    containing the `forecast_horizon`
    future values of the timeseries for each node.

    Args:
        data_array: np.ndarray with shape `(num_time_steps, num_routes)`
        input_sequence_length: Length of the input sequence (in number of timesteps).
        forecast_horizon: If `multi_horizon=True`, the target will be the values of the timeseries for 1 to
            `forecast_horizon` timesteps ahead. If `multi_horizon=False`, the target will be the value of the
            timeseries `forecast_horizon` steps ahead (only one value).
        batch_size: Number of timeseries samples in each batch.
        shuffle: Whether to shuffle output samples, or instead draw them in chronological order.
        multi_horizon: See `forecast_horizon`.

    Returns:
        A tf.data.Dataset instance.
    """

    inputs = timeseries_dataset_from_array(
        data_array[:-forecast_horizon],
        None,
        sequence_length=input_sequence_length,
        shuffle=False,
        batch_size=batch_size,
    )

    dataset = inputs
    target_offset = (
        input_sequence_length
        if multi_horizon
        else input_sequence_length + forecast_horizon - 1
    )
    target_seq_length = forecast_horizon if multi_horizon else 1
    targets = timeseries_dataset_from_array(
        target_array[target_offset:],
        None,
        sequence_length=target_seq_length,
        shuffle=False,
        batch_size=batch_size,
    )

    dataset = tf.data.Dataset.zip((inputs, targets))
    if shuffle:
        dataset = dataset.shuffle(100)

    return dataset.prefetch(16).cache()


train_dataset = create_tf_dataset(
    Xtrain,
    Ytrain,
    input_sequence_length,
    forecast_horizon,
    batch_size=batch_size,
    shuffle=False,
    multi_horizon=multi_horizon,
)

val_dataset = create_tf_dataset(
    Xval,
    Yval,
    input_sequence_length,
    forecast_horizon,
    batch_size=batch_size,
    shuffle=False,
    multi_horizon=multi_horizon,
)
test_dataset = create_tf_dataset(
    Xtest,
    Ytest,
    input_sequence_length,
    forecast_horizon,
    batch_size=batch_size,
    shuffle=False,
    multi_horizon=multi_horizon,
)

print(train_dataset)


# Adjacency Matrix

It was our original intent to use dynamic graphs for our st-gnn, however due to lack of time we kept it as a weighted average of the distance based distance_weight_adjacency() and an average of the adjancy matrices based on flights getAdjacencyMatrix().

In [ ]:
distance_adj = distance_weight_adjacency(airports, threshold=400)
# unfortunately we only work with static graphs for now
# We take the average of all the dynamic adjacencies from the function above (stored for brevity)
flight_adj = np.mean(getAdjacencyMatrix(airports, start, end, timeslotLength), axis=0)

# print(flight_adj)

# flight_adj = flight_adj_avg

adjacency_matrix = distance_adj * 0.4 + (1 - 0.4) * flight_adj

node_indices, neighbor_indices = np.where(adjacency_matrix != 0)

graph_edges = (node_indices.tolist(), neighbor_indices.tolist())
graph_num_nodes = adjacency_matrix.shape[0]

print(flight_adj.shape)
# print(graph_edges)
print(graph_num_nodes)


This block shows a heatmap of the adjacency matrix

In [ ]:
plt.imshow(adjacency_matrix)
plt.xticks(range(len(airports)), airports, fontsize=12, rotation=-90)
plt.yticks(range(len(airports)), airports, fontsize=12, rotation=0)
plt.show()


## Graph Convolution Layer

This is our GCN layer. Retrieved from a keras example and modified: https://keras.io/examples/timeseries/timeseries_traffic_forecasting/

In [ ]:
class GraphConv(layers.Layer):
    def __init__(
        self,
        in_feat,
        out_feat,
        # graph_info: GraphInfo,
        graph_num_nodes=graph_num_nodes,
        graph_edges: tuple = graph_edges,
        aggregation_type="mean",
        combination_type="concat",
        activation: typing.Optional[str] = None,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.in_feat = in_feat
        self.out_feat = out_feat

        self.graph_edges = graph_edges
        self.graph_num_nodes = graph_num_nodes

        self.aggregation_type = aggregation_type
        self.combination_type = combination_type
        self.weight = tf.Variable(
            initial_value=keras.initializers.glorot_uniform()(
                shape=(in_feat, out_feat), dtype="float32"
            ),
            trainable=True,
        )
        self.activation = layers.Activation(activation)

    def aggregate(self, neighbour_representations: tf.Tensor):
        aggregation_func = {
            "sum": tf.math.unsorted_segment_sum,
            "mean": tf.math.unsorted_segment_mean,
            "max": tf.math.unsorted_segment_max,
        }.get(self.aggregation_type)

        if aggregation_func:
            return aggregation_func(
                neighbour_representations,
                self.graph_edges[0],
                num_segments=self.graph_num_nodes,
            )

        raise ValueError(f"Invalid aggregation type: {self.aggregation_type}")

    def compute_nodes_representation(self, features: tf.Tensor):
        """Computes each node's representation.

        The nodes' representations are obtained by multiplying the features tensor with
        `self.weight`. Note that
        `self.weight` has shape `(in_feat, out_feat)`.

        Args:
            features: Tensor of shape `(num_nodes, batch_size, input_seq_len, in_feat)`

        Returns:
            A tensor of shape `(num_nodes, batch_size, input_seq_len, out_feat)`
        """
        return tf.matmul(features, self.weight)

    def compute_aggregated_messages(self, features: tf.Tensor):
        neighbour_representations = tf.gather(features, self.graph_edges[1])
        aggregated_messages = self.aggregate(neighbour_representations)
        return tf.matmul(aggregated_messages, self.weight)

    def update(self, nodes_representation: tf.Tensor, aggregated_messages: tf.Tensor):
        if self.combination_type == "concat":
            h = tf.concat([nodes_representation, aggregated_messages], axis=-1)
        elif self.combination_type == "add":
            h = nodes_representation + aggregated_messages
        else:
            raise ValueError(f"Invalid combination type: {self.combination_type}.")

        return self.activation(h)

    def call(self, features: tf.Tensor):
        """Forward pass.

        Args:
            features: tensor of shape `(num_nodes, batch_size, input_seq_len, in_feat)`

        Returns:
            A tensor of shape `(num_nodes, batch_size, input_seq_len, out_feat)`
        """
        nodes_representation = self.compute_nodes_representation(features)
        aggregated_messages = self.compute_aggregated_messages(features)
        return self.update(nodes_representation, aggregated_messages)


## LSTM including graph convolution layer

This is our complete model, it is based off a road-traffic example that predicts all features for the next step. We modified it heavily to allow for predictions of 2 labels and multihorizon predictions. Retrieved from a keras example and modified: https://keras.io/examples/timeseries/timeseries_traffic_forecasting/

In [ ]:
class LSTMGC(layers.Layer):
    """Layer comprising a convolution layer followed by LSTM and dense layers."""

    def __init__(
        self,
        in_feat,
        out_feat,
        lstm_units: int,
        input_seq_len: int,
        graph_edges: tuple,
        graph_num_nodes: int,
        graph_conv_params: typing.Optional[dict] = None,
        num_labels: int = 2,
        multi_horizon: int = True,
        forecast_horizon: int = forecast_horizon,
        **kwargs,
    ):

        if multi_horizon:
            self.forecast_horizon = forecast_horizon
        else:
            self.forecast_horizon = 1

        self.in_feat = in_feat
        self.out_feat = out_feat
        self.lstm_units = lstm_units
        self.input_seq_len = input_seq_len
        self.graph_edges = (graph_edges,)
        self.graph_num_nodes = (graph_num_nodes,)

        self.graph_conv_params = graph_conv_params
        self.multi_horizon = multi_horizon
        self.num_labels = num_labels
        super().__init__(**kwargs)

        # graph conv layer
        if graph_conv_params is None:
            graph_conv_params = {
                "aggregation_type": "mean",
                "combination_type": "concat",
                "activation": None,
            }

        # Layer definitions
        self.graph_conv = GraphConv(in_feat, out_feat, **graph_conv_params)
        # self.graph_conv2 = GraphConv(in_feat, out_feat, self.graph_edges, self.graph_num_nodes, **graph_conv_params)
        l2_reg = 2.5e-4  # L2 regularization rate

        self.lstm1 = layers.LSTM(
            lstm_units,
            return_sequences=True,
            activation="tanh",
            dropout=0.2,
            kernel_regularizer=l2(l2_reg),
            activity_regularizer=l2(l2_reg),
            bias_regularizer=l2(l2_reg),
        )
        self.lstm2 = layers.LSTM(
            lstm_units,
            activation="tanh",
            dropout=0.2,
            kernel_regularizer=l2(l2_reg),
            activity_regularizer=l2(l2_reg),
            bias_regularizer=l2(l2_reg),
        )
        # self.dense = layers.Dense(output_seq_len)
        self.denseThick = layers.Dense(128)
        self.denseThick2 = layers.Dense(64)
        self.denseThick3 = layers.Dense(16)
        self.dense = layers.Dense(self.forecast_horizon * self.num_labels)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "in_feat": self.in_feat,
                "out_feat": self.out_feat,
                "lstm_units": self.lstm_units,
                "input_seq_len": self.input_seq_len,
                "graph_conv_params": self.graph_conv_params,
                "graph_edges": self.graph_edges,
            }
        )
        return config

    def call(self, inputs):
        """Forward pass.

        Args:
            inputs: tf.Tensor of shape `(batch_size, input_seq_len, num_nodes, in_feat)`

        Returns:
            A tensor of shape `(batch_size, output_seq_len, num_nodes)`.
        """

        # convert shape to  (num_nodes, batch_size, input_seq_len, in_feat)
        inputs = tf.transpose(inputs, [2, 0, 1, 3])
        gcn_out = self.graph_conv(inputs)

        # print(f"The GCN output shape  = {gcn_out}")
        shape = tf.shape(gcn_out)
        num_nodes, batch_size, input_seq_len, out_feat = (
            shape[0],
            shape[1],
            shape[2],
            shape[3],
        )
        # LSTM takes only 3D tensors as input
        gcn_out = tf.reshape(gcn_out, (batch_size * num_nodes, input_seq_len, out_feat))
        # print(f"The input shape for the LSTM is {gcn_out}")
        lstmLayer1 = self.lstm1(
            gcn_out
        )  # lstm_out has shape: (batch_size * num_nodes, lstm_units)
        lstmLayer2 = self.lstm2(lstmLayer1)
        dense_1 = self.denseThick(lstmLayer2)
        dense_2 = self.denseThick2(dense_1)
        dense_3 = self.denseThick3(dense_2)

        dense_output = self.dense(dense_3)
        # dense_output has shape: (batch_size * num_nodes, multi_thing*2)

        output = tf.reshape(
            dense_output,
            (num_nodes, batch_size, self.forecast_horizon, self.num_labels),
        )
        final = tf.transpose(output, [1, 2, 0, 3])
        return final
        # # returns Tensor of shape (batch_size, forecast_horizon, num_nodes, nlabels)


## Compiling the model

In [ ]:
in_feat = Xarray.shape[2]

out_feat = 15
lstm_units = 128
graph_conv_params = {
    "aggregation_type": "mean",
    "combination_type": "concat",
    "activation": None,
}

st_gcn = LSTMGC(
    in_feat=in_feat,
    out_feat=out_feat,
    lstm_units=lstm_units,
    input_seq_len=input_sequence_length,
    graph_edges=graph_edges,
    graph_num_nodes=graph_num_nodes,
    graph_conv_params=graph_conv_params,
    num_labels=2,
    multi_horizon=multi_horizon,
    forecast_horizon=forecast_horizon,
)


inputs = layers.Input((input_sequence_length, graph_num_nodes, in_feat))
outputs = st_gcn(inputs)

model = keras.models.Model(inputs, outputs)

optimizer = Adam(learning_rate=learning_rate)
model.compile(
    optimizer=optimizer,
    loss=MeanSquaredError(reduction="auto", name="mean_absolute_error"),
    weighted_metrics=["acc"],
)

model.summary()


## Training the model

Training the model, if loading the model from a previous run was not requestd. Using tensorboard, training can be followed live.

In [ ]:
if not loadModel:
    log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, histogram_freq=1
    )

    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=epochs,
        callbacks=[
            keras.callbacks.EarlyStopping(patience=patience),
            tensorboard_callback,
        ],
    )


### Save model

In [ ]:
if saveModel and not loadModel:
    model.save(modelSaveLocation)
elif saveModel and loadModel:
    raise Exception(
        "You have enabled both loading and saving, which are incompatible. Model has not been saved"
    )


In [ ]:
if not loadModel:
    plt.plot(history.history["loss"], label="Train Loss")
    plt.plot(history.history["val_loss"], label="Validation Loss")
    plt.legend()
    plt.show()


#### Load model

In [ ]:
if loadModel:
    model = keras.models.load_model(modelSaveLocation)
    model.summary()
else:
    model = model


## Analize model performance on the test data

This part of the notebook gives the performance per forecast horizon in the first plot. In the second plot, the predicted airport delay over time is compared to the real delay over time to illustrate the accuracy of the model. The default setting displays a window of 100 hours. As the model has a multi-horizon feature, you can choose for how many hours into the future to predict. The model is trained to predict between 1 and 10 hours into the future. You can choose to display a bigger window with the windowSize feature and how many hours ahead predicted should be illustrated.

In [ ]:
windowSize = 200  # 1-400

forecastlen = 10  # 1-10
forecastlen = min(forecastlen, forecast_horizon)
print(f"Hours ahead = {forecastlen / (60/timeslotLength)}")


In [ ]:
def getLabelArrays(hour, input_sequence_length=input_sequence_length):
    syncindex = input_sequence_length + hour - 1
    yTestPred = model.predict(test_dataset, verbose=0)
    ypredFull = yTestPred[:windowSize, hour - 1, :, :]
    yactualFull = Ytest[syncindex : windowSize + syncindex :, :]

    mae1 = mean_absolute_error(ypredFull[:, :, 0], yactualFull[:, :, 0])
    mae2 = mean_absolute_error(ypredFull[:, :, 1], yactualFull[:, :, 1])
    rmse1 = np.sqrt(mean_squared_error(ypredFull[:, :, 0], yactualFull[:, :, 0]))
    rmse2 = np.sqrt(mean_squared_error(ypredFull[:, :, 1], yactualFull[:, :, 1]))
    r2_1 = r2_score(ypredFull[:, :, 0], yactualFull[:, :, 0])
    r2_2 = r2_score(ypredFull[:, :, 1], yactualFull[:, :, 1])

    time = testTimes[syncindex : windowSize + syncindex :]
    return ypredFull, yactualFull, mae1, mae2, rmse1, rmse2, r2_1, r2_2, time


def plotComparison(airport_index, hour):
    ypredFull, yactualFull, _, _, _, _, _, _, time = getLabelArrays(hour)
    fig, axs = plt.subplots(2, 1, sharex=True, num=airport_index)
    axs[0].plot(
        time, yactualFull[:, 0 + airport_index, 0], label="Actual Arrival Delay"
    )
    axs[1].plot(time, yactualFull[:, airport_index, 1], label="Actual Departure Delay")
    axs[0].plot(
        time, ypredFull[:, 0 + airport_index, 0], label="Predicted Arrival Delay"
    )
    axs[1].plot(time, ypredFull[:, airport_index, 1], label="Predicted Departure Delay")
    axs[0].legend()
    # axs[1].legend()
    axs[1].set_xlabel("Time (hours)")
    axs[0].set_ylabel("Arrival Delay (mins)")
    axs[1].set_ylabel("Departure Delay (mins)")
    plt.suptitle(
        f"Comparison for: {airports[airport_index]}. Forward: {hour / (60/timeslotLength)}h, Backward: {input_sequence_length / (60/timeslotLength)}h"
    )
    axs[1].xaxis.set_major_locator(mdates.HourLocator(interval=6))
    axs[1].xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
    plt.xticks(rotation=45)
    fig.set_figwidth(12)
    fig.set_figheight(6)


def plotErrorVsLookback(forecast_horizon=forecast_horizon, save=True, saveFolder="GNNPredictions"):
    fig2, axss = plt.subplots(3, 1, sharex=True, num=100)
    idxs = list(range(1, forecast_horizon + 1))
    lookforwards = [x/(60/timeslotLength) for x in range(1, forecast_horizon+1)]
    maeList = []
    maeList2 = []
    rmseList = []
    rmseList2 = []
    r2List = []
    r2List2 = []

    for h in idxs:
        ypredFull, yactualFull, mae1, mae2, rmse1, rmse2, r2_1, r2_2, time = getLabelArrays(h)
        maeList.append(mae1)
        maeList2.append(mae2)
        rmseList.append(rmse1)
        rmseList2.append(rmse2)
        r2List.append(r2_1)
        r2List2.append(r2_2)


    
    axss[0].plot(lookforwards, maeList, label = "Arrival Delay", marker="o")
    axss[0].plot(lookforwards, maeList2, label = "Departure Delay", marker="o")
    axss[1].plot(lookforwards, rmseList, label = "Arrival Delay", marker="o")
    axss[1].plot(lookforwards, rmseList2, label = "Departure Delay", marker="o")
    axss[2].plot(lookforwards, r2List, label = "Arrival Delay", marker="o")
    axss[2].plot(lookforwards, r2List2, label = "Departure Delay", marker="o")

    axss[0].set_ylabel("MAE (minutes)")
    axss[1].set_ylabel("RMSE (minutes)")
    axss[2].set_ylabel("R2 Score")
    # axss[0].grid()
    # axss[1].grid()
    # axss[2].grid()

    axss[2].set_xlabel("Look forward (hours)")

    axss[0].legend()

    fig2.set_figwidth(8)
    fig2.set_figheight(12)

plotErrorVsLookback()
plt.show()

for airportidx in range(0, int(len(airports)/5)):
    plotComparison(airportidx, forecastlen)
# plt.show()


# Save to csv
def savePredictions(saveFolder = "predictions"):
    print("Saving to CSV")
    idxs = list(range(1, forecast_horizon + 1))
    lookforwards = [x/(60/timeslotLength) for x in range(1, forecast_horizon+1)]
    maxlen = len(Ytest)

    for idx, lookforward in zip(idxs, lookforwards):
        syncindex = input_sequence_length + idx - 1
        yTestPred = model.predict(test_dataset, verbose=0)
        ypredFull = yTestPred[:maxlen-syncindex, idx - 1, :, :]

        yactualFull = Ytest[syncindex:maxlen - idxs[-1] + idx:, :]

        time = testTimes[syncindex:maxlen - idxs[-1] + idx]
        # print(ypredFull.shape, yactualFull.shape, len(time))

        P = pd.DataFrame()
        P["time"] = time
        # Double for loop here unfortunately ~fix later~
        for idd, airport in enumerate(airports):
            P[f"{airport}_arr_actual"] = yactualFull[:, idd, 0]
            P[f"{airport}_arr_predicted"] = ypredFull[:, idd, 0]
            P[f"{airport}_dep_actual"] = yactualFull[:, idd, 1]
            P[f"{airport}_dep_predicted"] = ypredFull[:, idd, 1]
        # print(P)
        savelocation = saveFolder + "/" + f"{lookforward}h.csv"
        P.to_csv(savelocation)


savePredictions()

## Generate data for Kepler gl visualization
If wanted, the data and predictions can be converted to the right format for use with kepler.gl. Kepler is the visualisation tool used for our report cover and in our presentation

In [ ]:
# syncindex = input_sequence_length + hour - 1
# yTestPred = model.predict(test_dataset, verbose=0)
# ypredFull = yTestPred[:, hour - 1, :, :]
# timeKepler = testTimes[syncindex::]
# Ykepler = Ytest[syncindex:, :]
# generateKeplerData(
#     airports=airports,
#     start=timeKepler[0],
#     end=end,
#     timeslotLength=timeslotLength,
#     predictions=ypredFull,
#     actual=Ykepler,
# )


## Add logbook to tensorboard
If wanted, the training logbook can be saved to Tensorboard.dev

In [ ]:
# !tensorboard dev upload --logdir ./logs \
#   --name "Tuning model 2" \
#   --description "Trying some stuff ans last run on TOP50"\
#   --one_shot
